<a href="https://colab.research.google.com/github/partizanos/master_thesis_unservised_anomaly_detection_jet_physics/blob/main/Observation_space_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import time 
print(time.ctime())
from google.colab import drive
drive.mount('/content/drive')


if os.getcwd() != '/content/drive/My Drive/Code/master_thesis/data_delphes' :
  os.chdir('/content/drive/My Drive/Code/master_thesis/data_delphes')
else: 
  print('Failed to find folder')
st= time.time()
print(time.ctime())


import time; st = time.ctime()
print(st )
!pip install imageio
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
# 'Not colab environment'
import tensorflow as tf
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
from IPython import display
import numpy as np
import matplotlib as plt
import pandas as pd 
! pip install torch torchvision
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import os
import h5py
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input,Dense,LSTM,Concatenate,RepeatVector,Subtract,BatchNormalization
from tensorflow.keras.optimizers import Adam,Nadam
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import h5py
import numpy as np
import h5py
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
print(time.ctime())
from sklearn.metrics import roc_curve, auc
! pip freeze > req.txt


# 1st Eta 
# 2nd Phi
# 3r Pt
# 4rth E

path_to_inputs = "./"
path_topjet = path_to_inputs+"final_tt_500kEvents.h5"
path_qcd = path_to_inputs+"final_jj_1MEvents.h5"
path_bb = path_to_inputs+"final_bb_500kEvents.h5"

# f = h5py.File(path_topjet,'r')
# jets = f['objects/jets/constituents'][:,0]
# fjet = f['objects/jets/obs'][:,0]
# f, jets.shape, fjet.shape
# f.keys(), f['high_level'].keys(), f['high_level/HT'].shape

print(time.ctime())

def ExyzToEtaPhiPtE(fourvectors):
    print("Inputs", fourvectors.shape)
    
    nconst = fourvectors.shape[1]
    pt = np.sqrt(np.sum(np.power(fourvectors[:,:,1:3],2),axis=-1))
    p =  np.sqrt(np.sum(np.power(fourvectors[:,:,1:],2),axis=-1))
    
    eta = -0.5*np.log( (1 - fourvectors[:,:,3] / p)/(1+fourvectors[:,:,3] / p)) 
    
    eta[(fourvectors[:,:,-1].mask) | (fourvectors[:,:,-1] == 0)] = 0
    phi = np.arctan2(fourvectors[:,:,2], fourvectors[:,:,1])
    e = fourvectors[:,:,0]
    
    phi = np.where(phi > np.pi, phi - 2*np.pi, phi)
    phi = np.where(phi <= np.pi, phi + 2*np.pi, phi)

    return np.ma.concatenate([eta.reshape(-1,nconst,1),phi.reshape(-1,nconst,1),pt.reshape(-1,nconst,1),e.reshape(-1,nconst,1)],axis=-1)

def transform_jets(jets,jet_pt,jet_e):
    #centre jet on leading pT cluster
    jet_eta = jets[:,0,0]
    jet_phi = jets[:,0,1]

    mask = jets.mask.copy()
    nconst = jets.shape[1]

    #jets = np.ma.masked_where(df.mask,jets)
    jets[:,:,:2] = np.ma.masked_array(
        np.concatenate([
                       (jets[:,:,0] - jet_eta.reshape(-1,1)).reshape(-1,nconst,1),
                       (jets[:,:,1] - jet_phi.reshape(-1,1)).reshape(-1,nconst,1)], 
                       axis=-1
                       ),
        mask=mask[:,:,:2]
    )

    #constraint phi between -pi and pi
    jets[:,:,1] = np.where(jets[:,:,1] < np.pi, jets[:,:,1] + 2*np.pi,jets[:,:,1])
    jets[:,:,1] = np.where(jets[:,:,1] >= np.pi, jets[:,:,1] - 2*np.pi,jets[:,:,1])
    
    #Add fraction pT/E
    fracs = np.concatenate([(jets[:,:,2] / jet_pt.reshape(-1,1)).reshape(-1,nconst,1), (jets[:,:,3] / jet_e.reshape(-1,1)).reshape(-1,nconst,1)],axis=-1)
    fracs = np.ma.masked_array(fracs,mask=mask[:,:,:2])
    jets = np.ma.concatenate([jets,fracs],axis=-1)

    #Rotate subleading cluster
    alpha = - np.arctan2(jets[:,1,1],jets[:,1,0])
    alpha = np.repeat(alpha,nconst).reshape(-1,nconst)
    jetseta = jets[:,:,0]*np.cos(alpha) - jets[:,:,1]*np.sin(alpha)
    jetsphi = jets[:,:,0]*np.sin(alpha) + jets[:,:,1]*np.cos(alpha)

    #Calculate Centre of P and flip so always positive
    flip = np.sum((jetsphi * jets[:,:,-2]),axis=-1) < 0
    jetsphi = np.ma.where(np.repeat(flip,nconst).reshape(-1,nconst), -1*jetsphi, jetsphi)
    jets[:,:,:2] = np.ma.concatenate([jetseta.reshape(-1,nconst,1),jetsphi.reshape(-1,nconst,1)],axis=-1)

    return jets

def getJetsFromFlat(path,key,nconst,label='is_signal_new'):
    nclusttot = 200 # total number of clusters, needed to get jet E and jet pT

    #Open dataset
    df = pd.read_hdf(path,key)
    #Get class labels
    if label is not None:
        labels = df[label].values

    #Get clusters and reshape, set masked entries
    df = df[df.columns[0:4*nclusttot]].values.reshape(-1,nclusttot,4)
    df = np.ma.masked_where(df == [0,0,0,0],df)

    #Get jet properties
    df2 = np.sum(df,axis=1)
    jet_pt = np.sqrt(np.sum(np.power(df2[:,1:3],2),axis=-1))
    jet_e = df2[:,0]

    #select clusters
    df = df[:,:nconst,:]

    #Convert to EtaPtPhiE
    jets = ExyzToEtaPhiPtE(df)
    jets = np.ma.masked_where(df.mask,jets)
    jets = transform_jets(jets,jet_pt,jet_e)

    if label is not None:
        return jets, labels
    else:
        return jets

def getJetsFromH5(filename,nconst=40,nattr=5,append_subleading=True,return_fjets=False):
    f = h5py.File(filename,'r')
    jets = f['objects/jets/constituents'][:,0]
    fjet = f['objects/jets/obs'][:,0]
    if append_subleading is True:
        jets = np.concatenate([jets,f['objects/jets/constituents'][:,1]])
        fjet = np.concatenate([fjet,f['objects/jets/obs'][:,1]])

    for _ in map(lambda x: x.resize(nattr*nconst,refcheck=False),jets): pass
    #while map(lambda x: x.resize(nattr*nconst,refcheck=False),jets).__next__(): pass
    jets = np.vstack(jets).reshape(-1,nconst,nattr)
    jets = jets[:,:,:4]

    def EfromPtEtaPhiM(arr):
        pt  = arr[0]
        eta = arr[1]
        phi = arr[2]
        m   = arr[3]
        return np.sqrt((pt*np.cos(phi))**2 + (pt*np.sin(phi))**2 + (pt*np.sinh(eta))**2 - m*m)
    
    jet_e = np.apply_along_axis(EfromPtEtaPhiM,1,fjet)
    #jet_e = fjet[:,0].reshape(-1,1,1)
    #fjet = ExyzToEtaPhiPtE(fjet)
    jet_pt = fjet[:,0].reshape(-1,1,1)    

    jets = np.ma.masked_where(jets == [0,0,0,0],jets)
    jets = ExyzToEtaPhiPtE(jets)
    jets = transform_jets(jets,jet_pt,jet_e)

    return (jets,fjet) if return_fjets else jets

print('edited at ', time.ctime())
print(time.ctime())
print('###############################')
top="./final_tt_500kEvents.h5"

path_to_inputs = './'
size_topjets=80000
size_qcdjets=80000

path_topjet = path_to_inputs+"final_tt_500kEvents.h5"
path_qcd = path_to_inputs+"final_jj_1MEvents.h5"

np.random.seed(23)
tf.random.set_seed(3)
output = 0 # used for schemas generation
start= time.time()
nclusters = 20
ncategories = 10
nlatent_dim =8
hd1=256
hd2=128
dense=64
batch_size = 512
nepochs = 70
# nepochs = 20
lr=5e-5
loss_w = [0.98,0.01,0.01]
print(time.ctime(),"############### READ TOP and qcd ")
print('read top',path_topjet,nclusters)

topjets_f = getJetsFromH5(path_topjet,nclusters,append_subleading=False)
print('read qcd',path_qcd,nclusters)
qcdjets_f = getJetsFromH5(path_qcd,nclusters,append_subleading=False)
print(topjets_f.shape, qcdjets_f.shape)
print(time.ctime())


Mon Oct 12 14:38:47 2020
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mon Oct 12 14:38:47 2020
Mon Oct 12 14:38:47 2020
Mon Oct 12 14:39:01 2020
Mon Oct 12 14:39:02 2020
edited at  Mon Oct 12 14:39:02 2020
Mon Oct 12 14:39:02 2020
###############################
Mon Oct 12 14:39:02 2020 ############### READ TOP and qcd 
read top ./final_tt_500kEvents.h5 20
Inputs (320124, 20, 4)
read qcd ./final_jj_1MEvents.h5 20
